In [ ]:
#!/usr/bin/env python3
# ===============================================
# PP_19 - MONTE CARLO MATCH SIMULATION (GOD SOTA 2026)
# TennisTitan - Point-by-Point Markov Chain Simulation
# ===============================================
#
# OBJECTIF:
# Simuler des matchs point-par-point pour obtenir:
# - P(3-0), P(3-1), P(3-2) pour Best of 5
# - P(2-0), P(2-1) pour Best of 3
# - Expected total games
# - Probability distributions pour betting (over/under, handicaps)
#
# MODÈLE MARKOV:
# États: (sets_A, sets_B, games_A, games_B, points_A, points_B, server)
# Transitions basées sur P(point gagné au service)
#
# Input: models/god_sota_2026/ (modèle entraîné)
# Output: predictions/monte_carlo/
# ===============================================

import numpy as np
import polars as pl
from pathlib import Path
from datetime import datetime
from dataclasses import dataclass
from typing import Tuple, Dict, List, Optional
import json
import joblib
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# ===============================================
# CONFIGURATION
# ===============================================
ROOT = Path(r"C:\Users\Administrateur\Tennis POLAR v2")
DATA_DIR = ROOT / "data_clean" / "ml_final"
MODELS_DIR = ROOT / "models" / "god_sota_2026"
OUTPUT_DIR = ROOT / "predictions" / "monte_carlo"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Simulation parameters
N_SIMULATIONS = 10000  # Nombre de simulations par match
TIEBREAK_THRESHOLD = 6  # Score pour tiebreak (6-6)
FINAL_SET_TIEBREAK = True  # Tiebreak en set décisif (style US Open)

print("=" * 70)
print("   PP_19 - MONTE CARLO MATCH SIMULATION (GOD SOTA 2026)")
print("=" * 70)
print(f"   {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"   Simulations per match: {N_SIMULATIONS:,}")
print("=" * 70)


# ===============================================
# TENNIS MATCH STATE
# ===============================================

@dataclass
class MatchState:
    """État complet d'un match de tennis."""
    sets_A: int = 0
    sets_B: int = 0
    games_A: int = 0
    games_B: int = 0
    points_A: int = 0  # 0, 1, 2, 3 = 0, 15, 30, 40
    points_B: int = 0
    server: str = "A"  # "A" ou "B"
    is_tiebreak: bool = False
    tb_points_A: int = 0
    tb_points_B: int = 0
    best_of: int = 3  # 3 ou 5 sets
    
    def copy(self):
        return MatchState(
            self.sets_A, self.sets_B, self.games_A, self.games_B,
            self.points_A, self.points_B, self.server, self.is_tiebreak,
            self.tb_points_A, self.tb_points_B, self.best_of
        )
    
    def winner(self) -> Optional[str]:
        """Retourne le gagnant ou None si match en cours."""
        sets_to_win = 3 if self.best_of == 5 else 2
        if self.sets_A >= sets_to_win:
            return "A"
        if self.sets_B >= sets_to_win:
            return "B"
        return None
    
    def total_games(self) -> int:
        """Nombre total de jeux joués."""
        # Note: on devrait tracker les jeux par set, mais pour simplifier
        # on calcule approximativement
        return self.games_A + self.games_B


# ===============================================
# POINT PROBABILITY MODEL
# ===============================================

class PointProbabilityModel:
    """
    Modèle de probabilité de gagner un point au service.
    
    Basé sur les statistiques de service du joueur et de retour de l'adversaire.
    """
    
    def __init__(self):
        # Default serve/return probabilities (ATP averages)
        self.default_serve_win = 0.64  # P(serveur gagne le point)
        self.default_first_serve_in = 0.62
        self.default_first_serve_win = 0.73
        self.default_second_serve_win = 0.52
    
    def get_point_prob(self, server: str, p_match_A: float,
                       serve_stats_A: Dict = None, serve_stats_B: Dict = None,
                       is_tiebreak: bool = False, pressure_factor: float = 1.0) -> float:
        """
        Calcule P(serveur gagne le point).
        
        Args:
            server: "A" ou "B"
            p_match_A: Probabilité que A gagne le match (du modèle ML)
            serve_stats_A: Stats de service de A (optionnel)
            serve_stats_B: Stats de service de B (optionnel)
            is_tiebreak: Si c'est un tiebreak (moins de variance)
            pressure_factor: Facteur de pression (1.0 = normal)
        
        Returns:
            P(serveur gagne le point)
        """
        
        # Méthode 1: Dérivation depuis p_match
        # Si p_match_A = 0.65, on peut estimer les probas de points
        # En utilisant un modèle inverse simplifié
        
        # Formule empirique: p_serve ≈ 0.5 + (p_match - 0.5) * k
        # où k ajuste la "force" du serveur
        k_serve = 0.25  # Coefficient empirique
        
        if server == "A":
            base_prob = self.default_serve_win + (p_match_A - 0.5) * k_serve
        else:
            # Pour B, on inverse
            p_match_B = 1 - p_match_A
            base_prob = self.default_serve_win + (p_match_B - 0.5) * k_serve
        
        # Ajustement tiebreak (légèrement plus serré)
        if is_tiebreak:
            base_prob = 0.5 + (base_prob - 0.5) * 0.9
        
        # Ajustement pression (break points, etc.)
        base_prob = 0.5 + (base_prob - 0.5) * pressure_factor
        
        # Clip pour éviter les valeurs extrêmes
        return np.clip(base_prob, 0.35, 0.85)
    
    def get_point_prob_from_stats(self, server: str,
                                   serve_pct_A: float, serve_pct_B: float,
                                   return_pct_A: float, return_pct_B: float) -> float:
        """
        Version alternative utilisant les stats de service/retour directement.
        
        Args:
            serve_pct_A: % points gagnés au service par A
            serve_pct_B: % points gagnés au service par B
            return_pct_A: % points gagnés en retour par A
            return_pct_B: % points gagnés en retour par B
        
        Returns:
            P(serveur gagne le point)
        """
        
        if server == "A":
            # A sert contre B
            # Combiner serve de A et return de B
            p_serve = serve_pct_A if serve_pct_A else self.default_serve_win
            p_return_opp = return_pct_B if return_pct_B else (1 - self.default_serve_win)
            
            # Moyenne pondérée (serve plus important)
            return 0.7 * p_serve + 0.3 * (1 - p_return_opp)
        else:
            p_serve = serve_pct_B if serve_pct_B else self.default_serve_win
            p_return_opp = return_pct_A if return_pct_A else (1 - self.default_serve_win)
            
            return 0.7 * p_serve + 0.3 * (1 - p_return_opp)


# ===============================================
# MATCH SIMULATOR
# ===============================================

class TennisMatchSimulator:
    """
    Simulateur de match de tennis point par point.
    
    Utilise une chaîne de Markov pour simuler chaque point
    et propager jusqu'à la fin du match.
    """
    
    def __init__(self, point_model: PointProbabilityModel = None):
        self.point_model = point_model or PointProbabilityModel()
        
        # Point values pour conversion
        self.point_names = {0: "0", 1: "15", 2: "30", 3: "40"}
    
    def simulate_point(self, state: MatchState, p_serve_wins: float) -> MatchState:
        """Simule un seul point et retourne le nouvel état."""
        
        state = state.copy()
        
        # Qui gagne le point?
        server_wins = np.random.random() < p_serve_wins
        
        if state.is_tiebreak:
            return self._simulate_tiebreak_point(state, server_wins)
        else:
            return self._simulate_normal_point(state, server_wins)
    
    def _simulate_normal_point(self, state: MatchState, server_wins: bool) -> MatchState:
        """Simule un point dans un jeu normal."""
        
        if server_wins:
            if state.server == "A":
                state.points_A += 1
            else:
                state.points_B += 1
        else:
            if state.server == "A":
                state.points_B += 1
            else:
                state.points_A += 1
        
        # Check for game won
        state = self._check_game_won(state)
        
        return state
    
    def _check_game_won(self, state: MatchState) -> MatchState:
        """Vérifie si le jeu est gagné et met à jour l'état."""
        
        # Deuce situation (40-40 ou plus)
        if state.points_A >= 3 and state.points_B >= 3:
            diff = state.points_A - state.points_B
            if diff >= 2:
                # Server wins game (if A) or returner (if B was serving)
                state = self._game_won(state, "A" if state.server == "A" else "B")
            elif diff <= -2:
                state = self._game_won(state, "B" if state.server == "A" else "A")
            # Sinon, deuce/advantage continue
        elif state.points_A >= 4:
            state = self._game_won(state, "A" if state.server == "A" else "B")
        elif state.points_B >= 4:
            state = self._game_won(state, "B" if state.server == "A" else "A")
        
        return state
    
    def _game_won(self, state: MatchState, winner: str) -> MatchState:
        """Met à jour l'état quand un jeu est gagné."""
        
        if winner == "A":
            state.games_A += 1
        else:
            state.games_B += 1
        
        # Reset points
        state.points_A = 0
        state.points_B = 0
        
        # Change server
        state.server = "B" if state.server == "A" else "A"
        
        # Check for set won
        state = self._check_set_won(state)
        
        return state
    
    def _check_set_won(self, state: MatchState) -> MatchState:
        """Vérifie si le set est gagné."""
        
        # Tiebreak condition
        if state.games_A == TIEBREAK_THRESHOLD and state.games_B == TIEBREAK_THRESHOLD:
            # Check if final set and no tiebreak rule
            is_final_set = (state.sets_A == state.sets_B == (state.best_of // 2))
            if is_final_set and not FINAL_SET_TIEBREAK:
                # Pas de tiebreak en set décisif (style Wimbledon ancien)
                pass
            else:
                state.is_tiebreak = True
                state.tb_points_A = 0
                state.tb_points_B = 0
            return state
        
        # Normal set win (6-X with 2 game lead, or 7-5)
        if state.games_A >= 6 and state.games_A - state.games_B >= 2:
            state = self._set_won(state, "A")
        elif state.games_B >= 6 and state.games_B - state.games_A >= 2:
            state = self._set_won(state, "B")
        
        return state
    
    def _set_won(self, state: MatchState, winner: str) -> MatchState:
        """Met à jour l'état quand un set est gagné."""
        
        if winner == "A":
            state.sets_A += 1
        else:
            state.sets_B += 1
        
        # Reset games
        state.games_A = 0
        state.games_B = 0
        state.is_tiebreak = False
        
        return state
    
    def _simulate_tiebreak_point(self, state: MatchState, server_wins: bool) -> MatchState:
        """Simule un point dans un tiebreak."""
        
        if server_wins:
            if state.server == "A":
                state.tb_points_A += 1
            else:
                state.tb_points_B += 1
        else:
            if state.server == "A":
                state.tb_points_B += 1
            else:
                state.tb_points_A += 1
        
        # Check for tiebreak won (7+ with 2 point lead)
        if state.tb_points_A >= 7 and state.tb_points_A - state.tb_points_B >= 2:
            state = self._set_won(state, "A")
        elif state.tb_points_B >= 7 and state.tb_points_B - state.tb_points_A >= 2:
            state = self._set_won(state, "B")
        
        # Change server every 2 points in tiebreak (after first point)
        total_tb_points = state.tb_points_A + state.tb_points_B
        if total_tb_points == 1 or (total_tb_points > 1 and (total_tb_points - 1) % 2 == 0):
            state.server = "B" if state.server == "A" else "A"
        
        return state
    
    def simulate_match(self, p_match_A: float, best_of: int = 3,
                       serve_stats: Dict = None) -> Dict:
        """
        Simule un match complet.
        
        Args:
            p_match_A: Probabilité que A gagne (du modèle ML)
            best_of: 3 ou 5 sets
            serve_stats: Stats de service optionnelles
        
        Returns:
            Dict avec résultats: winner, score, total_games, etc.
        """
        
        state = MatchState(best_of=best_of)
        games_history = []  # Pour tracker les jeux par set
        current_set_games = {"A": 0, "B": 0}
        
        max_points = 1000  # Sécurité contre boucle infinie
        point_count = 0
        
        while state.winner() is None and point_count < max_points:
            # Calculer proba de point
            p_serve = self.point_model.get_point_prob(
                state.server, p_match_A,
                is_tiebreak=state.is_tiebreak
            )
            
            # Stocker games avant le point
            games_before = (state.games_A, state.games_B)
            sets_before = (state.sets_A, state.sets_B)
            
            # Simuler le point
            state = self.simulate_point(state, p_serve)
            point_count += 1
            
            # Tracker les jeux
            if state.games_A != games_before[0] or state.games_B != games_before[1]:
                current_set_games["A"] = state.games_A
                current_set_games["B"] = state.games_B
            
            # Nouveau set?
            if state.sets_A != sets_before[0] or state.sets_B != sets_before[1]:
                games_history.append(current_set_games.copy())
                current_set_games = {"A": 0, "B": 0}
        
        # Calculer score final
        total_games = sum(g["A"] + g["B"] for g in games_history)
        
        # Format score
        score_str = " ".join([f"{g['A']}-{g['B']}" for g in games_history])
        
        return {
            "winner": state.winner(),
            "sets_A": state.sets_A,
            "sets_B": state.sets_B,
            "total_games": total_games,
            "score": score_str,
            "games_history": games_history,
            "points_played": point_count,
        }
    
    def monte_carlo_simulation(self, p_match_A: float, best_of: int = 3,
                                n_simulations: int = N_SIMULATIONS) -> Dict:
        """
        Lance N simulations et agrège les résultats.
        
        Returns:
            Dict avec distributions de probabilités
        """
        
        results = {
            "A_wins": 0,
            "B_wins": 0,
            "score_probs": {},  # "3-0", "3-1", etc.
            "total_games": [],
            "A_sets_when_win": [],
            "B_sets_when_win": [],
        }
        
        for _ in range(n_simulations):
            sim = self.simulate_match(p_match_A, best_of)
            
            if sim["winner"] == "A":
                results["A_wins"] += 1
                score_key = f"{sim['sets_A']}-{sim['sets_B']}"
                results["A_sets_when_win"].append(sim["sets_B"])
            else:
                results["B_wins"] += 1
                score_key = f"{sim['sets_B']}-{sim['sets_A']}"  # Du point de vue du gagnant
                results["B_sets_when_win"].append(sim["sets_A"])
            
            # Score probability
            full_score = f"{sim['sets_A']}-{sim['sets_B']}"
            results["score_probs"][full_score] = results["score_probs"].get(full_score, 0) + 1
            
            results["total_games"].append(sim["total_games"])
        
        # Normaliser
        n = n_simulations
        
        # Probabilités de score
        for key in results["score_probs"]:
            results["score_probs"][key] /= n
        
        # Stats sur les jeux
        games_array = np.array(results["total_games"])
        
        return {
            "p_A_wins": results["A_wins"] / n,
            "p_B_wins": results["B_wins"] / n,
            "score_probabilities": results["score_probs"],
            "expected_total_games": float(games_array.mean()),
            "std_total_games": float(games_array.std()),
            "median_total_games": float(np.median(games_array)),
            "percentiles_games": {
                "p10": float(np.percentile(games_array, 10)),
                "p25": float(np.percentile(games_array, 25)),
                "p50": float(np.percentile(games_array, 50)),
                "p75": float(np.percentile(games_array, 75)),
                "p90": float(np.percentile(games_array, 90)),
            },
            "p_over_X_games": {
                f"over_{x}": float((games_array > x).mean())
                for x in [20, 22, 24, 26, 28, 30, 32, 35, 38, 40]
            },
            "n_simulations": n_simulations,
        }


# ===============================================
# BETTING FEATURES
# ===============================================

def compute_betting_features(mc_results: Dict, best_of: int = 3) -> Dict:
    """
    Calcule les features utiles pour le betting depuis les résultats Monte Carlo.
    
    Returns:
        Dict avec probas pour différents marchés
    """
    
    score_probs = mc_results["score_probabilities"]
    
    features = {
        # Match winner
        "p_A_wins": mc_results["p_A_wins"],
        "p_B_wins": mc_results["p_B_wins"],
        
        # Total games
        "expected_games": mc_results["expected_total_games"],
        "games_std": mc_results["std_total_games"],
    }
    
    if best_of == 3:
        # Best of 3: scores possibles 2-0, 2-1, 0-2, 1-2
        features["p_2-0_A"] = score_probs.get("2-0", 0)
        features["p_2-1_A"] = score_probs.get("2-1", 0)
        features["p_0-2_A"] = score_probs.get("0-2", 0)
        features["p_1-2_A"] = score_probs.get("1-2", 0)
        
        # Handicap sets
        features["p_A_wins_-1.5_sets"] = features["p_2-0_A"]  # A gagne 2-0
        features["p_A_wins_+1.5_sets"] = mc_results["p_A_wins"]  # A gagne (any)
        features["p_B_wins_-1.5_sets"] = features["p_0-2_A"]  # B gagne 2-0
        features["p_B_wins_+1.5_sets"] = mc_results["p_B_wins"]  # B gagne (any)
        
    else:  # Best of 5
        # Best of 5: scores possibles 3-0, 3-1, 3-2, 0-3, 1-3, 2-3
        features["p_3-0_A"] = score_probs.get("3-0", 0)
        features["p_3-1_A"] = score_probs.get("3-1", 0)
        features["p_3-2_A"] = score_probs.get("3-2", 0)
        features["p_0-3_A"] = score_probs.get("0-3", 0)
        features["p_1-3_A"] = score_probs.get("1-3", 0)
        features["p_2-3_A"] = score_probs.get("2-3", 0)
        
        # Handicap sets
        features["p_A_wins_-1.5_sets"] = features["p_3-0_A"] + features["p_3-1_A"]
        features["p_A_wins_-2.5_sets"] = features["p_3-0_A"]
        features["p_A_wins_+1.5_sets"] = mc_results["p_A_wins"]
        features["p_A_wins_+2.5_sets"] = mc_results["p_A_wins"] + features["p_1-3_A"] + features["p_2-3_A"]
    
    # Over/Under games (common lines)
    for line in [20.5, 21.5, 22.5, 23.5, 24.5, 25.5]:
        features[f"p_over_{line}_games"] = mc_results["p_over_X_games"].get(f"over_{int(line)}", 
            float((np.array([mc_results["expected_total_games"]]) > line).mean()))
    
    return features


# ===============================================
# MAIN PROCESSING
# ===============================================

def process_predictions(predictions_file: Path = None) -> pl.DataFrame:
    """
    Charge les prédictions du modèle et calcule les features Monte Carlo.
    """
    
    print("\n[1/4] Loading predictions...")
    
    # Load test predictions or use provided file
    if predictions_file and predictions_file.exists():
        df = pl.read_parquet(predictions_file)
    else:
        # Try to load from standard location
        test_path = DATA_DIR / "test.parquet"
        if test_path.exists():
            df = pl.read_parquet(test_path)
        else:
            raise FileNotFoundError("No predictions file found!")
    
    print(f"  Loaded {len(df):,} matches")
    
    # Check for probability column
    prob_col = None
    for candidate in ["prob_A_wins", "prediction", "p_A", "odds_implied_prob_A"]:
        if candidate in df.columns:
            prob_col = candidate
            break
    
    if prob_col is None:
        print("  ⚠️ No probability column found, using default 0.5")
        df = df.with_columns([pl.lit(0.5).alias("prob_A_wins")])
        prob_col = "prob_A_wins"
    
    print(f"  Using probability column: {prob_col}")
    
    # Check for best_of column
    best_of_col = None
    for candidate in ["best_of_ta", "best_of", "bestof"]:
        if candidate in df.columns:
            best_of_col = candidate
            break
    
    if best_of_col is None:
        print("  ⚠️ No best_of column found, assuming Best of 3")
        df = df.with_columns([pl.lit(3).alias("best_of")])
        best_of_col = "best_of"
    
    return df, prob_col, best_of_col


def run_monte_carlo_batch(df: pl.DataFrame, prob_col: str, best_of_col: str,
                          n_simulations: int = N_SIMULATIONS) -> pl.DataFrame:
    """
    Exécute les simulations Monte Carlo pour tous les matchs.
    """
    
    print(f"\n[2/4] Running Monte Carlo simulations ({n_simulations:,} per match)...")
    
    simulator = TennisMatchSimulator()
    results = []
    
    # Convert to list for iteration
    matches = df.select([prob_col, best_of_col]).to_dicts()
    
    for i, match in enumerate(tqdm(matches, desc="Simulating")):
        p_A = match[prob_col]
        best_of = match[best_of_col]
        
        if p_A is None:
            p_A = 0.5
        if best_of is None or best_of not in [3, 5]:
            best_of = 3
        
        # Run simulation
        mc_results = simulator.monte_carlo_simulation(
            p_match_A=p_A,
            best_of=int(best_of),
            n_simulations=n_simulations
        )
        
        # Compute betting features
        betting_features = compute_betting_features(mc_results, int(best_of))
        
        # Combine results
        result = {
            "mc_p_A_wins": mc_results["p_A_wins"],
            "mc_expected_games": mc_results["expected_total_games"],
            "mc_std_games": mc_results["std_total_games"],
            "mc_median_games": mc_results["median_total_games"],
            "mc_p10_games": mc_results["percentiles_games"]["p10"],
            "mc_p90_games": mc_results["percentiles_games"]["p90"],
        }
        
        # Add score probabilities
        for score, prob in mc_results["score_probabilities"].items():
            result[f"mc_p_score_{score.replace('-', '_')}"] = prob
        
        # Add betting features
        for key, value in betting_features.items():
            result[f"bet_{key}"] = value
        
        results.append(result)
    
    # Convert to DataFrame
    mc_df = pl.DataFrame(results)
    
    print(f"  Generated {len(mc_df.columns)} Monte Carlo features")
    
    return mc_df


def main():
    """Pipeline complet Monte Carlo."""
    
    t0 = datetime.now()
    
    # Load data
    df, prob_col, best_of_col = process_predictions()
    
    # Run Monte Carlo
    mc_df = run_monte_carlo_batch(df, prob_col, best_of_col, N_SIMULATIONS)
    
    # Merge with original
    print("\n[3/4] Merging results...")
    
    # Add index for merge
    df = df.with_row_index("_idx")
    mc_df = mc_df.with_row_index("_idx")
    
    result_df = df.join(mc_df, on="_idx", how="left").drop("_idx")
    
    print(f"  Final shape: {result_df.shape}")
    
    # Save
    print("\n[4/4] Saving...")
    
    output_path = OUTPUT_DIR / "monte_carlo_predictions.parquet"
    result_df.write_parquet(output_path)
    print(f"  ✅ Saved: {output_path}")
    
    # Save summary stats
    summary = {
        "n_matches": len(result_df),
        "n_simulations_per_match": N_SIMULATIONS,
        "mc_features": [c for c in result_df.columns if c.startswith("mc_") or c.startswith("bet_")],
        "created": datetime.now().isoformat(),
    }
    
    summary_path = OUTPUT_DIR / "monte_carlo_summary.json"
    with open(summary_path, "w") as f:
        json.dump(summary, f, indent=2)
    
    elapsed = (datetime.now() - t0).total_seconds()
    
    print("\n" + "=" * 70)
    print("   ✅ PP_19 MONTE CARLO COMPLETE!")
    print("=" * 70)
    print(f"   ⏱️ Time: {elapsed/60:.1f} minutes")
    print(f"   📊 Matches: {len(result_df):,}")
    print(f"   📊 MC Features: {len(summary['mc_features'])}")
    print(f"   📁 Output: {OUTPUT_DIR}")
    print("""
📋 FEATURES CRÉÉES:

   == CORE ==
   • mc_p_A_wins              : P(A gagne) simulée
   • mc_expected_games        : Nombre de jeux attendus
   • mc_std_games             : Écart-type des jeux
   • mc_median_games          : Médiane des jeux
   • mc_p10_games, mc_p90_games : Percentiles 10/90
   
   == SCORE PROBABILITIES ==
   • mc_p_score_2_0, mc_p_score_2_1, etc.
   • mc_p_score_3_0, mc_p_score_3_1, mc_p_score_3_2 (Bo5)
   
   == BETTING MARKETS ==
   • bet_p_A_wins_-1.5_sets   : A gagne avec handicap -1.5 sets
   • bet_p_over_22.5_games    : Over 22.5 jeux
   • bet_expected_games       : Total games attendu

🔄 PROCHAINE ÉTAPE:
   PP_20 (Kelly Betting) pour le sizing optimal
""")
    
    return result_df


if __name__ == "__main__":
    main()